In [1]:
import pandas as pd
# from datetime import datetime
# from typing import Tuple
# import codecs, json
from greykite.common.data_loader import DataLoader
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster, ForecastResult
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

# Inputs

In [2]:
LOADS = 'out/loads.csv'

# (unix) timestamp of time offset
INITIAL_TIME = 1247270400

# time slot length
DELTA = 10 * 60
DELTA_ALIAS = '10T' # see: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases

# # adjusts flexibility of trend; increase to make more flexible; default is `0.05`
# CHANGEPOINT_PRIOR_SCALE = 0.05

# # number of samples of the predicted distribution
# NUMBER_OF_SAMPLES = 300

# number of predicted time slots
WINDOW = 100

# Preparation

In [3]:
l = pd.read_csv(LOADS)

In [4]:
metadata = MetadataParam(
    time_col="ds",
    value_col="y",
    freq=DELTA_ALIAS
)

## Preparing date times

In [5]:
def prepare_df(load_type: str) -> pd.DataFrame:
  df = l[[load_type]].rename(columns={load_type: 'y'})
  df['ds'] = df.index
  df['ds'] = (INITIAL_TIME + df['ds'] * DELTA).apply(lambda x: datetime.fromtimestamp(x))
  return df

## Making predictions

In [6]:
def predict(df: pd.DataFrame) -> ForecastResult:
  print(df)
  forecaster = Forecaster()
  return forecaster.run_forecast_config(
      df=df,
      config=ForecastConfig(
          model_template=ModelTemplateEnum.SILVERKITE.name,
          forecast_horizon=WINDOW,
          coverage=0.95,
          metadata_param=metadata
      )
  )

# Execution

In [7]:
predictions = {}

for load_type in l.columns:
  df = prepare_df(load_type)
  prediction = predict(df)
  predictions[load_type] = prediction
  summary = prediction.model[-1].summary(max_colwidth=30)
  print(summary)
  ts = prediction.timeseries
  ts.plot().show()
  backtest = prediction.backtest
  backtest.plot().show()

NameError: name 'datetime' is not defined

# Store predictions

Saves predictions as a JSON dump. For each load type the dump constains a list of predictions for each time slot (past and future). A prediction consists of a list of samples.

In [ ]:
json.dump({k: v.tolist() for k, v in predictions.items()}, codecs.open('out/predictions.json', 'w', encoding='utf-8'), separators=(',', ':'), sort_keys=True, indent=4)

AttributeError: 'ForecastResult' object has no attribute 'tolist'